# Other tables

## 0. Libraries

Install the necessary libraries.

In [1]:
import pandas as pd
import openpyxl

## 1. Load Data

Data from center and province of each row is loaded.

In [2]:
df = pd.read_pickle("./data/geocount_df_cleaned.pkl")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 37274 entries, 0 to 49968
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   province  37274 non-null  category
 1   center    37274 non-null  category
dtypes: category(2)
memory usage: 364.3 KB


## 2. Hospitals Table

A table with the amount of patients per center in the cohort is generated.

In [4]:
df_hospitals = pd.DataFrame(df['center'].value_counts()).sort_values(by=["count"], ascending=False)
df_hospitals.rename(columns={"count": "Count"}, inplace=True)
df_hospitals.index.name = "Center"
df_hospitals

,Count
Center,
H.U. Virgen del Rocío,5548
H.U. Virgen de la Victoria,3123
H.U. Virgen de las Nieves,3120
H.U. Reina Sofía,2486
H.U. San Cecilio,1984
H.U. Virgen Macarena,1966
H.U. Regional de Málaga,1768
H.U. Torrecárdenas,1760
H.U. Juan Ramón Jiménez,1416


In [5]:
save_path = "./TableCenters.html"

# # # # # # # # # # # # # # # # # # # # # # # # # # # 
pd.set_option('colheader_justify', 'center')   # FOR TABLE <th>

html_string = '''
<html>
  <head><title>HTML Pandas Dataframe with CSS</title></head>
  <link rel="stylesheet" type="text/css" href="df_style.css"/>
  <body>
    {table}
  </body>
</html>
'''

# OUTPUT AN HTML FILE
with open(save_path, 'w') as f:
    f.write(html_string.format(table=df_hospitals.to_html(classes='mystyle')))

Let's save as excel

In [6]:
excel_path = "../OUTPUT_figures_tables/Table_4_PatientesPerCenter.xlsx"
df_hospitals.to_excel(excel_path, sheet_name="Sheet1")

In [7]:
wb = openpyxl.load_workbook(excel_path)
sheet = wb["Sheet1"]
# Clean format
for row in sheet.iter_rows():
    for cell in row:
        cell.font = openpyxl.styles.Font()
        cell.border = openpyxl.styles.Border()
        cell.fill = openpyxl.styles.fills.PatternFill()

# Width Columns
sheet.column_dimensions['A'].width = 30
sheet.column_dimensions['B'].width = 10

#
# Align text to center in colum B
for row in sheet.iter_rows(min_row=1, min_col=2, max_col=2):
    for cell in row:
        cell.alignment = openpyxl.styles.Alignment(horizontal='right')

color_green1 = openpyxl.styles.colors.Color(rgb='007932')
color_green2 = openpyxl.styles.colors.Color(rgb='368f3f')
color_white = openpyxl.styles.colors.Color(rgb='ffffff')
color_gray1 = openpyxl.styles.colors.Color(rgb='c9c9c9')

if sheet['A1'].value == "Center":
    sheet.insert_rows(1)
    sheet['A1'] = "Num of patients per center"
sheet.merge_cells('A1:B1')
# Edit header
header_cell = sheet['A1']
sheet.row_dimensions[1].height=35
header_cell.alignment = openpyxl.styles.Alignment(horizontal='center', vertical='center')
header_cell.fill = openpyxl.styles.fills.PatternFill(patternType='solid', fgColor=color_green1)
header_cell.font = openpyxl.styles.Font(name="SourceSansPro", size=12, bold=True, color=color_white)

# Edit subheader cells
sheet.row_dimensions[2].height=20
for row in sheet['A2:B2']:
    for cell in row:
        cell.alignment = openpyxl.styles.Alignment(horizontal='center', vertical='center')
        cell.fill = openpyxl.styles.fills.PatternFill(patternType='solid', fgColor=color_green2)
        cell.font = openpyxl.styles.Font(name="SourceSansPro", size=12, bold=False, color=color_white)

# Edit variables rows (alterning row color)
cells_vars = []
for row in sheet[f'A3:A{sheet.max_row}']:
    for cell in row:
        if not isinstance(cell, openpyxl.cell.cell.MergedCell):
            cells_vars.append(cell)
for i, cell in enumerate(cells_vars):
    if i % 2 == 0:
        for row in sheet[f'A{cell.row}:B{cell.row}']:
            for cell in row:
                cell.fill = openpyxl.styles.fills.PatternFill(patternType='solid', fgColor=color_gray1)

#Set font all table_body
for row in sheet[f'A3:B{sheet.max_row}']:
    for cell in row:
        cell.font = openpyxl.styles.Font(name="SourceSansPro", size=10)

## Border 
border1 = openpyxl.styles.Border(bottom=openpyxl.styles.Side(border_style='thin', color='000000'))
side =openpyxl.styles.Side(border_style='thin', color='000000')

# header border
for row in sheet['A2:B2']:
    for cell in row:
        cell.border = border1

#Outborder left
for row in sheet[f'A1:A{sheet.max_row}']:
    for cell in row:
        prev_border = cell.border

        new_border = openpyxl.styles.Border(left=side,
                                            right=prev_border.right,
                                            top=prev_border.top,
                                            bottom=prev_border.bottom)
        
        cell.border = new_border

#Outborder bottom
for row in sheet[f'A{sheet.max_row}:B{sheet.max_row}']:
    for cell in row:
        prev_border = cell.border

        new_border = openpyxl.styles.Border(left=prev_border.left,
                                            right=prev_border.right,
                                            top=prev_border.top,
                                            bottom=side)
        
        cell.border = new_border

#Outborder right
for row in sheet[f'B{sheet.min_row}:B{sheet.max_row}']:
    for cell in row:
        prev_border = cell.border

        new_border = openpyxl.styles.Border(left=prev_border.left,
                                            right=side,
                                            top=prev_border.top,
                                            bottom=prev_border.bottom)
        
        cell.border = new_border        

#Outborder header
for row in sheet['A1:B1']:
    for cell in row:
        prev_border = cell.border

        new_border = openpyxl.styles.Border(left=prev_border.left,
                                            right=prev_border.right,
                                            top=side,
                                            bottom=prev_border.bottom)
        
        cell.border = new_border

wb.save(excel_path)

## 3. Province Table

A table with the amount of patients per province in the cohort is generated.

In [8]:
df_province = pd.DataFrame(df['province'].value_counts()).sort_values(by=["count"], ascending=False)
df_province.rename(columns={"count": "Count"}, inplace=True)
df_province.index.name = "Province"
df_province

,Count
Province,
Sevilla,9321
Málaga,6167
Granada,5954
Cádiz,4518
Córdoba,3653
Jaén,2674
Huelva,2629
Almería,2358


In [9]:
save_path = "./TableProvinces.html"

# # # # # # # # # # # # # # # # # # # # # # # # # # # 
pd.set_option('colheader_justify', 'center')   # FOR TABLE <th>

html_string = '''
<html>
  <head><title>HTML Pandas Dataframe with CSS</title></head>
  <link rel="stylesheet" type="text/css" href="df_style.css"/>
  <body>
    {table}
  </body>
</html>
'''

# OUTPUT AN HTML FILE
with open(save_path, 'w') as f:
    f.write(html_string.format(table=df_province.to_html(classes='mystyle')))

Let's save as excel

In [10]:
excel_path = "../OUTPUT_figures_tables/Table_5_PatientesPerProvince.xlsx"
df_province.to_excel(excel_path, sheet_name="Sheet1")

In [11]:
wb = openpyxl.load_workbook(excel_path)
sheet = wb["Sheet1"]
# Clean format
for row in sheet.iter_rows():
    for cell in row:
        cell.font = openpyxl.styles.Font()
        cell.border = openpyxl.styles.Border()
        cell.fill = openpyxl.styles.fills.PatternFill()

# Width Columns
sheet.column_dimensions['A'].width = 30
sheet.column_dimensions['B'].width = 10

#
# Align text to center in colum B
for row in sheet.iter_rows(min_row=1, min_col=2, max_col=2):
    for cell in row:
        cell.alignment = openpyxl.styles.Alignment(horizontal='right')

color_green1 = openpyxl.styles.colors.Color(rgb='007932')
color_green2 = openpyxl.styles.colors.Color(rgb='368f3f')
color_white = openpyxl.styles.colors.Color(rgb='ffffff')
color_gray1 = openpyxl.styles.colors.Color(rgb='c9c9c9')

if sheet['A1'].value == "Province":
    sheet.insert_rows(1)
    sheet['A1'] = "Num of patients per province"
sheet.merge_cells('A1:B1')
# Edit header
header_cell = sheet['A1']
sheet.row_dimensions[1].height=35
header_cell.alignment = openpyxl.styles.Alignment(horizontal='center', vertical='center')
header_cell.fill = openpyxl.styles.fills.PatternFill(patternType='solid', fgColor=color_green1)
header_cell.font = openpyxl.styles.Font(name="SourceSansPro", size=12, bold=True, color=color_white)

# Edit subheader cells
sheet.row_dimensions[2].height=20
for row in sheet['A2:B2']:
    for cell in row:
        cell.alignment = openpyxl.styles.Alignment(horizontal='center', vertical='center')
        cell.fill = openpyxl.styles.fills.PatternFill(patternType='solid', fgColor=color_green2)
        cell.font = openpyxl.styles.Font(name="SourceSansPro", size=12, bold=False, color=color_white)

# Edit variables rows (alterning row color)
cells_vars = []
for row in sheet[f'A3:A{sheet.max_row}']:
    for cell in row:
        if not isinstance(cell, openpyxl.cell.cell.MergedCell):
            cells_vars.append(cell)
for i, cell in enumerate(cells_vars):
    if i % 2 == 0:
        for row in sheet[f'A{cell.row}:B{cell.row}']:
            for cell in row:
                cell.fill = openpyxl.styles.fills.PatternFill(patternType='solid', fgColor=color_gray1)

#Set font all table_body
for row in sheet[f'A3:B{sheet.max_row}']:
    for cell in row:
        cell.font = openpyxl.styles.Font(name="SourceSansPro", size=10)

# Border 
border1 = openpyxl.styles.Border(bottom=openpyxl.styles.Side(border_style='thin', color='000000'))
side =openpyxl.styles.Side(border_style='thin', color='000000')

# header border
for row in sheet['A2:B2']:
    for cell in row:
        cell.border = border1

#Outborder left
for row in sheet[f'A1:A{sheet.max_row}']:
    for cell in row:
        prev_border = cell.border

        new_border = openpyxl.styles.Border(left=side,
                                            right=prev_border.right,
                                            top=prev_border.top,
                                            bottom=prev_border.bottom)
        
        cell.border = new_border

#Outborder bottom
for row in sheet[f'A{sheet.max_row}:B{sheet.max_row}']:
    for cell in row:
        prev_border = cell.border

        new_border = openpyxl.styles.Border(left=prev_border.left,
                                            right=prev_border.right,
                                            top=prev_border.top,
                                            bottom=side)
        
        cell.border = new_border

#Outborder right
for row in sheet[f'B{sheet.min_row}:B{sheet.max_row}']:
    for cell in row:
        prev_border = cell.border

        new_border = openpyxl.styles.Border(left=prev_border.left,
                                            right=side,
                                            top=prev_border.top,
                                            bottom=prev_border.bottom)
        
        cell.border = new_border        

#Outborder header
for row in sheet['A1:B1']:
    for cell in row:
        prev_border = cell.border

        new_border = openpyxl.styles.Border(left=prev_border.left,
                                            right=prev_border.right,
                                            top=side,
                                            bottom=prev_border.bottom)
        
        cell.border = new_border

wb.save(excel_path)